Because of the partial acquisitions, bbregister fails for some runs of some subjects. First shot at fixing it will be to re-run bbregister with the lyman registration outputs as an initial first guess


In [1]:
from moss.mosaic import Mosaic
import nibabel as nib
import multiprocessing
#os and i/o
import os
import numpy as np
import glob
from os.path import abspath
import csv
import shutil
import gc

In [2]:
#preliminary housekeeping
home_dir = '/data/home/iballard/mag/'
subj_file = home_dir + 'subjects.txt'
sub_list = list(np.loadtxt(subj_file,'string'))
os.chdir(home_dir)

In [21]:
#re-arrange directory structure
exp = 'mag'
coreg_files = ['func2anat.png','func2anat_tkreg.dat','func2anat_flirt.mat']
for sub in sub_list:
    for run in range(1,5):
        sub_path = home_dir + '/analysis/' + exp + '/' + sub + '/preproc/run_' + str(run) + '/'
        orig_coreg = sub_path + 'orig_coreg/'

        if os.path.exists(sub_path) and not os.path.exists(orig_coreg):
            os.mkdir(orig_coreg)

            for f in coreg_files:
                old = sub_path + f
                new = orig_coreg + f
                os.rename(old,new)

In [43]:
def rerun_coreg(in_tuple):
    sub, exp, run = in_tuple
    sub_path = home_dir + '/analysis/' + exp + '/' + sub + '/preproc/run_' + str(run) + '/'

    if os.path.exists(sub_path):    
        #skull strip mean_func
        mov_file = sub_path + 'mean_func.nii.gz' 
        new_mov_file = sub_path + 'mean_func_bet.nii.gz' 
#         os.remove(new_mov_file)
        cmd = ['bet2',mov_file,new_mov_file,'-f','.05']
        cmd = ' '.join(cmd)
        if not os.path.exists(new_mov_file):
            os.system(cmd)
        
        #run bbregister
        fs_dir = home_dir +'/data/' + sub
        init_reg = sub_path +  'orig_coreg/func2anat_tkreg.dat'
        out_reg = sub_path + 'func2anat_tkreg.dat'
        out_reg_fsl = sub_path + 'func2anat_flirt.mat'
        out_coreg = sub_path + '/coreg_mean_func.nii.gz'
        cmd = ['bbregister','--mov',new_mov_file,'--s',
               fs_dir,'--bold','--init-reg', init_reg, 
               '--reg',out_reg, '--fslmat',out_reg_fsl,
               '--o',out_coreg]
        cmd = ' '.join(cmd)
        print cmd
        if not os.path.exists(out_coreg):
            os.system(cmd)

        #save output
        out_png = sub_path + "func2anat.png"
        wm_file = fs_dir + '/mri/wm.mgz'
        wm_data = nib.load(wm_file).get_data().astype(bool).astype(int)
#         m = Mosaic(out_coreg, wm_data, step=3)
#         m.plot_contours(["#DD2222"])
#         m.savefig(out_png)
#         m.close()
rerun_coreg(('subject14','mag',4))

bbregister --mov /data/home/iballard/mag//analysis/mag/subject14/preproc/run_4/mean_func_bet.nii.gz --s /data/home/iballard/mag//data/subject14 --bold --init-reg /data/home/iballard/mag//analysis/mag/subject14/preproc/run_4/orig_coreg/func2anat_tkreg.dat --reg /data/home/iballard/mag//analysis/mag/subject14/preproc/run_4/func2anat_tkreg.dat --fslmat /data/home/iballard/mag//analysis/mag/subject14/preproc/run_4/func2anat_flirt.mat --o /data/home/iballard/mag//analysis/mag/subject14/preproc/run_4//coreg_mean_func.nii.gz


In [24]:
in_tuples = []
for exp in ['mag']:
    for sub in sub_list:
        for run in range(4,5):
            in_tuples.append((sub,exp,run))

In [25]:
pool = multiprocessing.Pool(processes = 20)
output = pool.map(rerun_coreg,in_tuples)
pool.terminate()
pool.join()

In [33]:
redo_tuples = []
for sub in sub_list:
    for run in range(1,5):
        sub_path = home_dir + '/analysis/' + exp + '/' + sub + '/preproc/run_' + str(run) + '/'
        f = sub_path + 'func2anat_tkreg.dat.mincost'
        if os.path.exists(f):
            cost = np.loadtxt(f)
            cost = cost[0]
            print cost
            if cost > .6:
                print sub, run
                redo_tuples.append((sub,exp,run))
        else:
            print 'nodata',sub,run

NameError: name 'exp' is not defined

In [44]:
#make image after hand tweaking troublesome registrations
for in_tuple in redo_tuples:
    #save output
    sub, exp, run = in_tuple
    sub_path = home_dir + '/analysis/' + exp + '/' + sub + '/preproc/run_' + str(run) + '/'
    fs_dir = home_dir +'/data/' + sub
    out_coreg = sub_path + '/coreg_mean_func.nii.gz'
        
    out_png = sub_path + "func2anat.png"
    wm_file = fs_dir + '/mri/wm.mgz'
    wm_data = nib.load(wm_file).get_data().astype(bool).astype(int)
    m = Mosaic(out_coreg, wm_data, step=3)
    m.plot_contours(["#DD2222"])
    m.savefig(out_png)
    m.close()

In [53]:
for sub in sub_list:
    for run in range(1,5):
        sub_path = home_dir + '/analysis/' + exp + '/' + sub + '/preproc/run_' + str(run) + '/'
        if os.path.exists(sub_path):
            mask = sub_path + '/functional_mask.nii.gz'
            mean = sub_path + '/mean_func.nii.gz'
            cmd = ['fslcpgeom',mean,mask]
            cmd = ' '.join(cmd)
            os.system(cmd)

In [20]:
#save bbregister params
bbregister_dir = home_dir + 'bbregister/'
for sub in sub_list:
    for run in range(1,5):
        run_dir = bbregister_dir + sub +'/' + 'run_' + str(run) + '/'
        analysis_dir = home_dir + '/analysis/mag/' + sub + '/preproc/run_' + str(run) + '/'
        if os.path.exists(analysis_dir):
            reg_files = glob.glob(analysis_dir + 'func2anat*')
            for f in reg_files:
                cmd = ['cp',f,run_dir]
                cmd = ' '.join(cmd)
                os.system(cmd)

In [32]:
#save bbregister params
bbregister_dir = home_dir + 'bbregister/'
for sub in sub_list:
    for run in range(1,5):
        run_dir = bbregister_dir + sub + '/' + 'run_' + str(run) + '/'
        data_reg_dir = home_dir + 'data/' + sub + '/reg/' + 'run_' + str(run)

        if os.path.exists(run_dir):
            
            reg_files = glob.glob(run_dir + 'func2anat*')
            for f in reg_files:
                cmd = ['cp',f,data_reg_dir]
                cmd = ' '.join(cmd)
#                 os.system(cmd)

In [24]:
data_dir = home_dir + 'data/'
for sub in sub_list:
    data_reg = data_dir + sub + '/reg/'
    for run in range(1,5):
        d = data_reg + 'run_' + str(run)
        os.mkdir(d)


In [4]:
data_dir = home_dir + 'analysis/mag/'
for sub in ['asu1']:
    data_reg = data_dir + sub + '/preproc/'
    for run in range(1,5):
        d = data_reg + 'run_' + str(run)
        mask= d +'/functional_mask.nii.gz'
        f = d + '/unsmoothed_timeseries.nii.gz'
        data = nib.load(f).get_data()
        mask = nib.load(mask).get_data()
        data = data[mask.astype(bool)]
        tsnr = data.mean(axis = 1) / data.std(axis = 1)
        print np.median(tsnr)


80.3173
91.5632
73.4163
85.9613


In [6]:
redo_tuples = []
exp = 'mag'
for sub in ['asu4','asu8','asu9']:
    for run in range(0,4):
        sub_path = home_dir + '/analysis/workingdir/mag/preproc/bbregister/_subject_id_' + \
            sub + '/func2anat/mapflow/_func2anat' + str(run) + '/'
        f = sub_path + 'func2anat_tkreg.dat.mincost'
        if os.path.exists(f):
            cost = np.loadtxt(f)
            cost = cost[0]
            if cost > .6:
                print sub, run,  cost
                redo_tuples.append((sub,exp,run))
        else:
            print 'nodata',sub,run

asu4 0 0.628112
asu4 1 0.629543
asu4 2 0.669468
asu4 3 0.651203
asu8 0 0.726286
asu8 1 0.725355
asu8 2 0.768752
asu8 3 0.771027
asu9 0 0.653548
asu9 1 0.667343
asu9 2 0.62086
